In [1]:
import os
import re
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

from utils import *

In [2]:
gim_path = '../logs_gim_so_verbose'
kis_path = '../logs_kis_so_verbose'
base_instances = {'Gim': [], 'Kis': []}
prepro_instances = {'Gim': [], 'Kis': []}

In [3]:
print(count_satwp_extractions_over_benchmarks(gim_path, 'Gim'))
print(count_satwp_extractions_over_benchmarks(kis_path, 'Kis'))

{'Gim_prepro': 237, 'Gim_base': 11}
{'Kis_prepro': 249, 'Kis_base': 18}


In [10]:
# override function to get names of instances
def count_satwp_extractions_over_benchmarks(path, prefix):
    global prepro_instances
    global base_instances
    prepro_instances[prefix] = []
    base_instances[prefix] = []
    dirs = next(os.walk(path))[1]
    h = {f"{prefix}_prepro": 0,
         f"{prefix}_base": 0}
    for name in dirs:
        file = open(f"{path}/{name}/out_file", 'r')
        log = file.read().__str__()
        res = get_satwp_extraction(log)
        h[f"{prefix}_prepro"] += res['prepro']
        h[f"{prefix}_base"] += res['base']
        if res['prepro']:
            prepro_instances[prefix] += [name]
        if res['base']:
            base_instances[prefix] += [name]

count_satwp_extractions_over_benchmarks(gim_path, 'Gim')
count_satwp_extractions_over_benchmarks(kis_path, 'Kis')

def print_list(list, offset):
    for e in list:
        print(f"{offset}{e}")

print('Solved with BASE instance:')
print('  Gim:')
print_list(base_instances['Gim'], '    ')
print('  Kis:')
print_list(base_instances['Kis'], '    ')

""" print('Solved with PREPRO instance:')
print('  Gim:')
print_list(prepro_instances['Gim'], '    ')
print('  Kis:')
print_list(prepro_instances['Kis'], '    ') """

Solved with BASE instance:
  Gim:
    rovers1_ks99i.renamed-as.sat05-3971.cnf
    g2-T99.2.0.cnf
    shift1add.28943.cnf
    TimetableCNFEncoding_20_UNKNOWN.cnf
    c499_gr_2pin_w6.shuffled.cnf
    mm-1x10-10-10-sb.1.shuffled-as.sat03-1489.cnf
    vmpc_24.cnf
    iso-ukn004.shuffled-as.sat05-3385.cnf
    iso-brn100.shuffled-as.sat05-3025.cnf
    ferry8_ks99i.renamed-as.sat05-4005.cnf
    vmpc_28.shuffled-as.sat05-1957.cnf
  Kis:
    g2-T99.2.0.cnf
    shift1add.28943.cnf
    ibm-2004-03-k70.cnf
    TimetableCNFEncoding_20_UNKNOWN.cnf
    brent_67_0.05.cnf
    brent_69_0.05.cnf
    c499_gr_2pin_w6.shuffled.cnf
    brent_69_0.cnf
    mm-1x10-10-10-sb.1.shuffled-as.sat03-1489.cnf
    mrpp_8x8#22_10.cnf
    c880_gr_rcs_w7.shuffled.cnf
    vmpc_24.cnf
    iso-ukn004.shuffled-as.sat05-3385.cnf
    SC23_Timetable_C_481_E_51_Cl_32_D_6_T_50.cnf
    iso-brn100.shuffled-as.sat05-3025.cnf
    brent_65_0.1.cnf
    vmpc_28.shuffled-as.sat05-1957.cnf
    SC23_Timetable_C_473_E_52_Cl_32_D_6_T_50.cnf


" print('Solved with PREPRO instance:')\nprint('  Gim:')\nprint_list(prepro_instances['Gim'], '    ')\nprint('  Kis:')\nprint_list(prepro_instances['Kis'], '    ') "

In [13]:
df = pd.read_csv('track_2023_database.csv', sep='\t')
df = df.drop(columns=['hash', 'isohash', 'author', 'proceedings', 'minisat1m', 'track'])
df = df.rename(columns={'filename': 'name'})
df = df.merge(pd.read_csv('benchmark_meta_data.csv', sep='\t'),
              how='inner',
              on='name')
df = df.merge(parse_overview(gim_path, 'gim'),
              how='inner',
              on='name')
df = df.sort_values('variables')
df

,family,result,name,variables,clauses,gim_result,gim_memory,gim_runtime
14,tseitin-formulas,unsat,Urquhart-s3-b3.shuffled-as.sat03-1556.cnf,45,376,-1,0.00,0.701362
141,pigeon-hole,unsat,php-010-009.shuffled-as.sat05-1185.cnf,90,415,-1,0.00,0.740708
398,tseitin-formulas,unsat,Urquhart-s5-b4.shuffled.cnf,114,888,0,1.11,0.000000
165,coloring,unsat,clqcolor-08-06-07.shuffled-as.sat05-1257.cnf,132,1527,-1,0.00,0.304089
339,pigeon-hole,unsat,tph8.cnf,136,5457,-1,1.19,201.350967
...,...,...,...,...,...,...,...,...
144,hashtable-safety,unsat,hash_table_find_safety_size_25.cnf,16012436,73097259,-1,73.60,37.233536
272,hashtable-safety,unsat,hash_table_find_safety_size_26.cnf,16990686,77502660,-1,78.57,31.720535
178,hashtable-safety,unsat,hash_table_find_safety_size_27.cnf,17997155,82032556,-1,83.13,45.223816
45,hashtable-safety,unsat,hash_table_find_safety_size_29.cnf,20094661,91465575,-1,92.68,46.707535


In [16]:
df.loc[df['name'].isin(base_instances['Gim'])]

,family,result,name,variables,clauses,gim_result,gim_memory,gim_runtime
117,planning,sat,rovers1_ks99i.renamed-as.sat05-3971.cnf,439,5423,1,0.00,0.036621
74,cryptography,sat,vmpc_24.cnf,576,67872,1,0.00,0.399208
6,cryptography,sat,vmpc_28.shuffled-as.sat05-1957.cnf,784,108080,1,0.00,1.012349
169,quasigroup-completion,sat,iso-ukn004.shuffled-as.sat05-3385.cnf,1890,12666,1,0.00,0.042835
244,fpga-routing,sat,c499_gr_2pin_w6.shuffled.cnf,2070,22470,1,0.00,0.167574
19,matrix-multiplication,sat,mm-1x10-10-10-sb.1.shuffled-as.sat03-1489.cnf,2396,83733,1,0.00,1.040454
167,planning,sat,ferry8_ks99i.renamed-as.sat05-4005.cnf,2547,32525,1,0.00,0.071375
199,quasigroup-completion,sat,iso-brn100.shuffled-as.sat05-3025.cnf,3588,10984,1,0.00,0.040262
109,scheduling,sat,TimetableCNFEncoding_20_UNKNOWN.cnf,378430,2649518,1,9.29,13.663494
193,bitvector,unsat,shift1add.28943.cnf,636737,1128758,-1,0.44,2.523799


In [17]:
df.loc[df['name'].isin(base_instances['Kis'])]

,family,result,name,variables,clauses,gim_result,gim_memory,gim_runtime
74,cryptography,sat,vmpc_24.cnf,576,67872,1,0.00,0.399208
6,cryptography,sat,vmpc_28.shuffled-as.sat05-1957.cnf,784,108080,1,0.00,1.012349
169,quasigroup-completion,sat,iso-ukn004.shuffled-as.sat05-3385.cnf,1890,12666,1,0.00,0.042835
244,fpga-routing,sat,c499_gr_2pin_w6.shuffled.cnf,2070,22470,1,0.00,0.167574
19,matrix-multiplication,sat,mm-1x10-10-10-sb.1.shuffled-as.sat03-1489.cnf,2396,83733,1,0.00,1.040454
199,quasigroup-completion,sat,iso-brn100.shuffled-as.sat05-3025.cnf,3588,10984,1,0.00,0.040262
143,fpga-routing,sat,c880_gr_rcs_w7.shuffled.cnf,4592,61745,1,0.00,0.061725
138,planning,unsat,mrpp_8x8#22_10.cnf,8918,72538,-1,0.00,0.206085
2,hardware-verification,sat,ibm-2004-03-k70.cnf,69839,286405,1,0.00,0.662141
371,brent-equations,sat,brent_65_0.1.cnf,77577,335642,1,0.00,0.886266
